# World Bank - Education Attainment Summarizing
Reduced '.csv' files from WB_data Notebook

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import requests
import matplotlib.pyplot as plt
from matplotlib import cbook
from scipy.stats import linregress
import scipy.stats as st
import numpy as np
import time


# Impor the OpenWeatherMap API key
from api_keys import geoapify_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [2]:
# WB data set
WB_africadata_2000_bins = Path("Resources\WB_africadata_2000_bins.csv")

In [3]:
# Read data file with the Pandas library
# encoding?, i.e encoding="ISO-8859-1"
WB_africadata_2000_df2 = pd.read_csv(WB_africadata_2000_bins)

In [4]:
WB_africadata_2000_df2

,series_id,country_code,country_name,year,value,Year Group
0,SH.HIV.INCD.TL,KEN,Kenya,2017,3.400000e+04,2016 - 2020
1,SH.HIV.INCD.TL,LBR,Liberia,2017,1.600000e+03,2016 - 2020
2,SH.HIV.INCD.TL,LSO,Lesotho,2017,1.000000e+04,2016 - 2020
3,SH.HIV.INCD.TL,MDG,Madagascar,2017,7.200000e+03,2016 - 2020
4,SH.HIV.INCD.TL,MOZ,Mozambique,2017,1.300000e+05,2016 - 2020
...,...,...,...,...,...,...
652157,VA.NO.SRC,WSM,Samoa,2006,3.000000e+00,2006 - 2010
652158,FM.LBL.BMNY.GD.ZS,TCD,Chad,2010,1.146678e+01,2006 - 2010
652159,NE.CON.PRVT.CN.AD,SDN,Sudan,2000,2.199464e+10,2000 - 2005
652160,NE.CON.PRVT.CN.AD,SDN,Sudan,2003,3.190503e+10,2000 - 2005


# Slicing of the Data
###In order to analyze the data and to understand the impact of the Foreign Aid with regards to poverty rate, literacy and mortality, the data was sliced by the "series_id". The deifinition of the diferent "series_id" values (indicators) is described in a second file downloaded from the Nasdaq API and saved as 'WB_metadata_df'.

###To facilitate/expedite the search and filtering of the relevant a function was created to search the text strings : "key_word". This function is case sensitive and it's used is combined with the review of the search output selection to identify the indicators that best match our work objective.

In [5]:
# WB id series definition data set
WB_metadata = Path("Resources/WB_METADATA_f7ce7fba293ccc6eb39cdf15fb097982.csv")

In [6]:
# Read data file with the Pandas library
WB_metadata_df = pd.read_csv(WB_metadata)

In [7]:
WB_metadata_df

,series_id,name,description
0,DC.DAC.DEUL.CD,"Net bilateral aid flows from DAC donors, Germa...",Net bilateral aid flows from DAC donors are th...
1,RQ.STD.ERR,Regulatory Quality: Standard Error,Regulatory Quality captures perceptions of the...
2,EG.USE.PCAP.KG.OE,Energy use (kg of oil equivalent per capita),Energy use refers to use of primary energy bef...
3,EN.POP.EL5M.UR.ZS,Urban population living in areas where elevati...,Urban population below 5m is the percentage of...
4,per_lm_alllm.cov_q1_tot,Coverage of unemployment benefits and ALMP in ...,Coverage of unemployment benefits and active l...
...,...,...,...
1479,SL.TLF.BASC.FE.ZS,"Labor force with basic education, female (% of...",The ratio of the labor force with basic educat...
1480,SL.GDP.PCAP.EM.KD,GDP per person employed (constant 2021 PPP $),GDP per person employed is gross domestic prod...
1481,SL.TLF.INTM.MA.ZS,"Labor force with intermediate education, male ...",The ratio of the labor force with intermediate...
1482,SL.UEM.NEET.FE.ZS,"Share of youth not in education, employment or...","Share of youth not in education, employment or..."


In [8]:
# This function will search within a text string for a given 'key word'. It is case sensitive

def key_word(df, search_col, word_txt, result_col):
    i=0   
    result_ls = []
    for item in df[search_col]:
        text_ls = [x.strip() for x in df[search_col][i].split()]
        if word_txt in text_ls:
            result_ls.append(df[result_col][i])
        i+=1
    if i >= df[search_col].count():
        if len(result_ls) > 0:
            return result_ls
        else:
            return print("Keyword Not found")

In [9]:
education_search = key_word(WB_metadata_df, 'name', 'Educational', 'series_id')

In [ ]:
#len(education_search)

In [10]:
education_search2= WB_metadata_df[WB_metadata_df['series_id'].isin(education_search)]

In [ ]:
#education_search2

# Selection of Indicators for Analysis

###Indicators are selected out of the search result and used to create redu ed data sets. In the case of the Mortality indicator, we selected three indicators : mortality under age 5-females only, mortality under age 5-males only, and mortality under age 5-combined. Values listed are the rate per 100,000 parts.

###In order to organize the data, we prepared 6 data sets. One for each indicator.

In [ ]:
# Literacy indicators : percentage of population
#SE.SEC.CUAT.LO.ZS - The percentage of population ages 25 and over that attained or completed lower secondary education.
#SE.SEC.CUAT.UP.ZS - The percentage of population ages 25 and over that attained or completed upper secondary education.
#SE.TER.CUAT.MS.ZS - The percentage of population ages 25 and over that attained or completed Master's or equivalent.

## Educational Attainment : Lower Secondary (% Pop.> 25yrs)

In [11]:
WB_Educ_LS_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SE.SEC.CUAT.LO.ZS"]
#WB_Educ_LS_df = WB_Educ_LS_df.rename(columns = {'value': 'Lower Secondary Educ. (% Pop.> 25yrs)'})
WB_Educ_LS_df

,series_id,country_code,country_name,year,value,Year Group
133129,SE.SEC.CUAT.LO.ZS,RWA,Rwanda,2010,12.582890,2006 - 2010
324759,SE.SEC.CUAT.LO.ZS,TZA,Tanzania,2002,5.986470,2000 - 2005
324760,SE.SEC.CUAT.LO.ZS,UGA,Uganda,2002,11.518880,2000 - 2005
324761,SE.SEC.CUAT.LO.ZS,ZWE,Zimbabwe,2002,46.016670,2000 - 2005
324762,SE.SEC.CUAT.LO.ZS,SLE,Sierra Leone,2003,16.482738,2000 - 2005
...,...,...,...,...,...,...
324894,SE.SEC.CUAT.LO.ZS,KEN,Kenya,2022,64.952110,2021 - 2023
324895,SE.SEC.CUAT.LO.ZS,NGA,Nigeria,2022,69.982292,2021 - 2023
324896,SE.SEC.CUAT.LO.ZS,RWA,Rwanda,2022,17.950159,2021 - 2023
324897,SE.SEC.CUAT.LO.ZS,SOM,Somalia,2022,11.111570,2021 - 2023


In [12]:
reduc_Educ_LS_df = WB_Educ_LS_df.reset_index()
reduc_Educ_LS_df.drop(columns=['index','series_id'], inplace=True)
reduc_Educ_LS_df

,country_code,country_name,year,value,Year Group
0,RWA,Rwanda,2010,12.582890,2006 - 2010
1,TZA,Tanzania,2002,5.986470,2000 - 2005
2,UGA,Uganda,2002,11.518880,2000 - 2005
3,ZWE,Zimbabwe,2002,46.016670,2000 - 2005
4,SLE,Sierra Leone,2003,16.482738,2000 - 2005
...,...,...,...,...,...
136,KEN,Kenya,2022,64.952110,2021 - 2023
137,NGA,Nigeria,2022,69.982292,2021 - 2023
138,RWA,Rwanda,2022,17.950159,2021 - 2023
139,SOM,Somalia,2022,11.111570,2021 - 2023


In [13]:
Educ_LS_table = reduc_Educ_LS_df.pivot(index='year', columns='country_name', values='value')
Educ_LS_table

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Ghana,Kenya,...,Senegal,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe
year,,,,,,,,,,,,,,,,,,,,,
2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.986470,NaN,NaN,11.518880,NaN,46.016670
2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,16.482738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,22.165590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,18.937050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,7.93816,NaN,NaN,NaN,44.430729,NaN,40.595158,NaN,...,7.547190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,2.03617,NaN,NaN,NaN,NaN,7.21509,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,13.809693,NaN,37.081161,23.627661,NaN,NaN
2009,10.561316,NaN,NaN,NaN,10.044390,NaN,NaN,NaN,NaN,32.911598,...,NaN,NaN,NaN,NaN,NaN,26.944210,NaN,NaN,NaN,NaN
2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.319618,29.073750,...,NaN,NaN,NaN,NaN,11.699676,NaN,39.334690,28.830111,NaN,NaN


## Educational Attainment : Upper Secondary (% Pop.> 25yrs)

In [14]:
WB_Educ_US_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SE.SEC.CUAT.UP.ZS"]
WB_Educ_US_df

,series_id,country_code,country_name,year,value,Year Group
60104,SE.SEC.CUAT.UP.ZS,WSM,Samoa,2001,63.037628,2000 - 2005
60105,SE.SEC.CUAT.UP.ZS,TZA,Tanzania,2002,1.622880,2000 - 2005
60106,SE.SEC.CUAT.UP.ZS,UGA,Uganda,2002,6.453940,2000 - 2005
60107,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2002,10.357510,2000 - 2005
60108,SE.SEC.CUAT.UP.ZS,SLE,Sierra Leone,2003,4.066010,2000 - 2005
...,...,...,...,...,...,...
149288,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2017,12.264890,2016 - 2020
149290,SE.SEC.CUAT.UP.ZS,KEN,Kenya,2019,33.388390,2016 - 2020
149292,SE.SEC.CUAT.UP.ZS,ZMB,Zambia,2021,36.321079,2021 - 2023
149293,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2021,64.132111,2021 - 2023


In [15]:
reduc_Educ_US_df = WB_Educ_US_df.reset_index()
reduc_Educ_US_df.drop(columns=['index','series_id'], inplace=True)
reduc_Educ_US_df

,country_code,country_name,year,value,Year Group
0,WSM,Samoa,2001,63.037628,2000 - 2005
1,TZA,Tanzania,2002,1.622880,2000 - 2005
2,UGA,Uganda,2002,6.453940,2000 - 2005
3,ZWE,Zimbabwe,2002,10.357510,2000 - 2005
4,SLE,Sierra Leone,2003,4.066010,2000 - 2005
...,...,...,...,...,...
133,ZWE,Zimbabwe,2017,12.264890,2016 - 2020
134,KEN,Kenya,2019,33.388390,2016 - 2020
135,ZMB,Zambia,2021,36.321079,2021 - 2023
136,ZWE,Zimbabwe,2021,64.132111,2021 - 2023


In [16]:
Educ_US_table = reduc_Educ_US_df.pivot(index='year', columns='country_name', values='value')
Educ_US_table

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Ghana,Kenya,...,Senegal,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe
year,,,,,,,,,,,,,,,,,,,,,
2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.622880,NaN,NaN,6.453940,NaN,10.357510
2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.066010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,6.494953,NaN,NaN,NaN,5.519980,NaN,NaN,NaN,NaN,NaN,...,NaN,12.129740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,4.00732,NaN,NaN,NaN,40.380260,NaN,12.395874,NaN,...,4.92511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,0.49510,NaN,NaN,NaN,NaN,7.024190,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.635780,NaN,NaN,8.773930,NaN,NaN
2009,7.719856,NaN,NaN,NaN,5.104420,NaN,NaN,NaN,NaN,27.056290,...,NaN,NaN,NaN,NaN,NaN,8.90192,NaN,NaN,NaN,NaN


In [ ]:
#WB_Educ_US_df = WB_Educ_US_df.rename(columns = {'value': 'Upper Secondary Educ. (% Pop.> 25yrs)'})
#WB_Educ_US_df

## Educational Attainment : Masters (% Pop.> 25yrs)

In [17]:
WB_Educ_MS_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SE.TER.CUAT.MS.ZS"]
WB_Educ_MS_df

,series_id,country_code,country_name,year,value,Year Group
72191,SE.TER.CUAT.MS.ZS,SLE,Sierra Leone,2003,0.210999,2000 - 2005
72192,SE.TER.CUAT.MS.ZS,AGO,Angola,2004,0.475335,2000 - 2005
72193,SE.TER.CUAT.MS.ZS,MWI,Malawi,2005,0.452600,2000 - 2005
72194,SE.TER.CUAT.MS.ZS,GHA,Ghana,2006,0.240368,2006 - 2010
72195,SE.TER.CUAT.MS.ZS,TZA,Tanzania,2008,0.128440,2006 - 2010
...,...,...,...,...,...,...
72281,SE.TER.CUAT.MS.ZS,ZMB,Zambia,2022,0.317543,2021 - 2023
149610,SE.TER.CUAT.MS.ZS,NGA,Nigeria,2006,1.642740,2006 - 2010
149611,SE.TER.CUAT.MS.ZS,TGO,Togo,2017,1.253972,2016 - 2020
149612,SE.TER.CUAT.MS.ZS,MRT,Mauritania,2019,1.938819,2016 - 2020


In [18]:
reduc_Educ_MS_df = WB_Educ_MS_df.reset_index()
reduc_Educ_MS_df.drop(columns=['index','series_id'], inplace=True)
reduc_Educ_MS_df

,country_code,country_name,year,value,Year Group
0,SLE,Sierra Leone,2003,0.210999,2000 - 2005
1,AGO,Angola,2004,0.475335,2000 - 2005
2,MWI,Malawi,2005,0.452600,2000 - 2005
3,GHA,Ghana,2006,0.240368,2006 - 2010
4,TZA,Tanzania,2008,0.128440,2006 - 2010
...,...,...,...,...,...
85,ZMB,Zambia,2022,0.317543,2021 - 2023
86,NGA,Nigeria,2006,1.642740,2006 - 2010
87,TGO,Togo,2017,1.253972,2016 - 2020
88,MRT,Mauritania,2019,1.938819,2016 - 2020


In [19]:
Educ_MS_table = reduc_Educ_MS_df.pivot(index='year', columns='country_name', values='value')
Educ_MS_table

country_name,Angola,Botswana,Burkina Faso,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Ghana,Kenya,Lesotho,...,Sao Tome and Principe,Senegal,Sierra Leone,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe
year,,,,,,,,,,,,,,,,,,,,,
2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.210999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,0.475335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.240368,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.128440,NaN,NaN,NaN,NaN,NaN
2009,3.022058,NaN,NaN,0.262690,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.502748,NaN,NaN,...,NaN,NaN,NaN,NaN,0.103473,NaN,NaN,NaN,NaN,NaN
2011,0.578822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.158080,NaN,0.879369,NaN,2.404216,NaN,NaN,NaN,NaN
2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,0.245774,NaN,NaN,0.360282,NaN,NaN


In [ ]:
#WB_Educ_MS_df = WB_Educ_MS_df.rename(columns = {'value': 'Masters Educ. (% Pop.> 25yrs)'})
#WB_Educ_MS_df

In [20]:
data5 = {
    '2000 Lower Secondary Educ. (% Pop.> 25yrs)': (Educ_LS_table.iloc[0, 0:31]),
    '2022 Lower Secondary Educ. (% Pop.> 25yrs)': (Educ_LS_table.iloc[-1, 0:31]),
    '2000 Upper Secondary Educ. (% Pop.> 25yrs)': (Educ_US_table.iloc[0, 0:31]),
    '2022 Upper Secondary Educ. (% Pop.> 25yrs)': (Educ_US_table.iloc[-1, 0:31]),
    '2000 Masters Educ. (% Pop.> 25yrs)': (Educ_MS_table.iloc[0, 0:31]),
    '2022 Masters Educ. (% Pop.> 25yrs)': (Educ_MS_table.iloc[-1, 0:31])
}
summary_table5 = pd.DataFrame(data5)
summary_table5


,2000 Lower Secondary Educ. (% Pop.> 25yrs),2022 Lower Secondary Educ. (% Pop.> 25yrs),2000 Upper Secondary Educ. (% Pop.> 25yrs),2022 Upper Secondary Educ. (% Pop.> 25yrs),2000 Masters Educ. (% Pop.> 25yrs),2022 Masters Educ. (% Pop.> 25yrs)
country_name,,,,,,
Angola,NaN,NaN,NaN,NaN,NaN,NaN
Botswana,NaN,70.004646,NaN,44.884315,NaN,1.215302
Burkina Faso,NaN,NaN,NaN,NaN,NaN,NaN
Central African Republic,NaN,NaN,NaN,NaN,NaN,NaN
Chad,NaN,NaN,NaN,NaN,NaN,NaN
Djibouti,NaN,NaN,NaN,NaN,NaN,NaN
"Egypt, Arab Rep.",NaN,NaN,NaN,NaN,NaN,NaN
Ethiopia,NaN,NaN,NaN,NaN,NaN,NaN
Ghana,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
summary_table5.to_csv("Resources\WB_education_summary.csv", encoding="utf-8", index=False)